# Welcome to Bioinformatics final project!

This **"tutorial colab notebook"** are created and organized by group 3: Christofer Julio and Zihao Chen.
Our task for the bioinformatics final project is to mimic the result of this paper:

```
Meng-Shin Shiao, Jia-Ming Chang, Wen-Lang Fan, Mei-Yeh Jade Lu, Cedric Notredame, Shu Fang, Rumi Kondo, Wen-Hsiung Li, Expression Divergence of Chemosensory Genes between Drosophila sechellia and Its Sibling Species and Its Implications for Host Shift , Genome Biology and Evolution, Volume 7, Issue 10, October 2015, Pages 2843–2858, https://doi.org/10.1093/gbe/evv183
```
However, due to the limitation of the resources, we are only able to replicate the **1 sample set each**.


**Advantages** in using colab notebook:


1.   First, colab notebook has its convenience way to install packages from various systems. In our case, we use bioconda and original python packages to process the data, and it works pretty well. However, remember to install everything first before we start.
2.   Second, colab works wonderful for some people who has low computing resource (old notebook/PC), as colab actually has 12 GB RAM, T4 GPU, and 100 GB disk space
3. Third, since we are not using any GPU resource, we can actually connect to colab for way longer time compared to people who intend to utilize colab for Deep learning.
4. You can run several codes simultaneously.

Now, here are some **tips**:


1.   Make sure to delete the runtime often, since processing big alignment file will take a lot of the disk space, and it can result in automatic stopping from the system once all the disk space are gone.
2.   Make sure you have a big google drive spaces, especially when you are dealing with big data. In our case, we need more than 1 TB of disk space if we use all the dataset (Google drive has 2TB subscription).
3. One of the main disadvantages of Google colab is that you need to upload the files here. And it will take a very long time for each file sometimes, especially when you don't have good internet connection.


**before we start, run this to download and configure conda environment.**







In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh && \
chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh && \
bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')


# install required packages and import the data from google drive


Our package mainly comes from bioconda (for fastq files processing, hisat2, cufflink, etc.), and some of them are imported from python packages (especially when doing plotting, creating dataframe, etc).

In addition, we need to mount google drive to google colab, since our fastq files are saved there.

In [ ]:
# fastqc is used for checking the quality of the sequence
!conda install -c bioconda fastqc -y

# trimomatic is used for trim the sequence
! conda install -c bioconda trimmomatic -y

# hisat2 for aligning sequence
# if it doesn't work, install conda hisat2 first, and uninstall conda hisat2. It suddenly works somehow.
!apt-get install hisat2

# install subreads for removing duplicates and feature counts
!conda create -n featurecounts -c bioconda subread -y

In [ ]:
# run this to install samsul

!apt-get install libssl1.0.0

# Download and install samtools
!wget https://github.com/samtools/samtools/releases/download/1.10/samtools-1.10.tar.bz2
!tar xjf samtools-1.10.tar.bz2
%cd samtools-1.10
!make
!make install

In [ ]:
# Install necessary packages
!pip install cufflinks
!apt-get install -y libhdf5-dev

# Download and install Cufflinks
!wget http://cole-trapnell-lab.github.io/cufflinks/assets/downloads/cufflinks-2.2.1.Linux_x86_64.tar.gz
!tar -zxvf cufflinks-2.2.1.Linux_x86_64.tar.gz
!mv cufflinks-2.2.1.Linux_x86_64/* /usr/local/bin/


In [ ]:
# import required python packages
from google.colab import drive
import os
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import numpy

In [ ]:
# mounting google colab drive
drive.mount('/content/drive')


Mounted at /content/drive


# Check your data with fastqc

Fastqc is used for checking the quality of the sequence. Bad sequence quality will affect the accuracy of the mapping, and the quality of the alignment in general. This is why checking the quality with Fastqc is an important step before we proceed to the next step.

In [ ]:
# put your file path in the file_path variable

def fastqc(file_path):
  !fastqc {file_path}

# run this code if you intend to check fastqc on all files:

def fastqc(file_path):
  for fasta in os.listdir(file_path):
    fasta_w_path = os.path.join(file_path, fasta)
    !fastqc {fasta_w_path}


# Using Trimmomatic to trim the "bad" sequences

As we have mentioned before, low-quality sequence can be bad for your overall research project. Hence, what should we do to improve our sequence quality? One of the most common way to make our sequence better is to trim the sequence.

Trimming low-quality sequence can be beneficial, especially to improve the quality of the data. It is also helped to decrease the probability of sequencing errors, PCR bias, or other factors such as poor sample quality.

In our case, after some trial and error, we decide to use this settings:

```
param = {'SLIDINGWINDOW':"4:20", "MINLEN": 60}
```




However, if the sequence is "good enough" according to Fastqc, we can skip this step.

In [ ]:
# trim sequences for all the _2 fasta files

def trim(path_source, path_dest, fastq_files_names, param):

  for fastq in fastq_files_names:

    # create path source and path destination
    path_source_names = os.path.join(path_source, fastq)
    path_dest_names = os.path.join(path_dest, fastq)

    # trimmomatic command
    !trimmomatic SE -threads 8 -phred33 {path_source_names} {path_dest_names} SLIDINGWINDOW:{param["SLIDINGWINDOW"]} MINLEN:{param["MINLEN"]}


In [ ]:
# declare path
path_source = r'/content/drive/MyDrive/fastaq/D_sim_jp'
path_dest = r'/content/drive/MyDrive/fastaq/D_sim_jp_trimmed'

# the second fastq files have bad sequences in general so we list it here
fastq_files_names = ["SRR1973492_2.fastq.gz", "SRR1973495_2.fastq.gz"]

# parameter for the trimming we use
param = {'SLIDINGWINDOW':"4:20", "MINLEN": 60}

trim(path_source, path_dest, fastq_files_names, param)

# Building Index, alignment with HISAT2

Then, once we have bunch of nice sequences, what we need to do next is to align the sequences with the reference genomes. Reference genomes, as its name implies, is used as the standardized genome for comparing it with the sampled genomes.

Hence, by aligning the sampled genomes with the standardized genomes, researchers will have a better understanding of the genetic variations, relationships, and similarities between the sampled genomes and the reference genomes.



In [ ]:
# # hisat2 build index

def hisat2_build(path_source, path_dest, files=list()):

  # check if the path exist
  os.makedirs(path_dest, exist_ok=True)

  path_source_names = os.path.join(path_source, files[0])
  path_dest_names = os.path.join(path_dest, files[1])
  !hisat2-build -p 8 {path_source_names} {path_dest_names}

  print('index building finished!')

In [ ]:
def hisat2_align(genome_index_folder, input_directory, output_directory):

    #  check if the path exist
    os.makedirs(output_directory, exist_ok=True)
    all_files = os.listdir(input_directory)

    # Filter for forward and reverse reads separately
    forward_reads = [file for file in all_files if file.endswith('_1.fastq.gz')]
    reverse_reads = [file for file in all_files if file.endswith('_2.fastq.gz')]
    print(reverse_reads)

    # Loop through each forward read
    for forward_read in forward_reads:
        forward_reads_path = os.path.join(input_directory, forward_read)
        filename, type1, type2 = forward_read.split('.')
        output_sam = os.path.join(output_directory, f"{filename}_alignment.sam")

        # HISAT2 command for forward read
        hisat2_command = f"hisat2 -x {genome_index_folder}/genome_index -U '{forward_reads_path}' -S '{output_sam}'"
        os.system(hisat2_command)

        print(f"HISAT2 alignment completed for {forward_read}")

    # Loop through each reverse read
    for reverse_read in reverse_reads:
        reverse_reads_path = os.path.join(input_directory, reverse_read)
        filename, type1, type2 = reverse_read.split('.')
        output_sam = os.path.join(output_directory, f"{filename}_alignment.sam")

        # HISAT2 command for reverse read
        hisat2_command = f"hisat2 -x {genome_index_folder}/genome_index -U '{reverse_reads_path}' -S '{output_sam}'"
        os.system(hisat2_command)

        print(f"HISAT2 alignment completed for {reverse_read}")

In [ ]:
path_source = r'/content/drive/MyDrive/fastaq/reference_genome'
path_dest = r'/content/drive/MyDrive/fastaq/reference_genome_index'
files=['dsim-all-chromosome-r1.3.fasta', 'genome_index']

hisat2_build(path_source, path_dest, files)

In [ ]:
genome_index_folder = r'/content/drive/MyDrive/fastaq/d_melan_alg_process/genome_index_melan'
input_directory = r'/content/drive/MyDrive/bio_dataset/dataset/D_melanogaster_Taiwan'
output_directory = r'/content/drive/MyDrive/fastaq/D_melan_aligned'

hisat2_align(genome_index_folder, input_directory, output_directory)

['SRR5638358_2.fastq.gz']
HISAT2 alignment completed for SRR5638358_2.fastq.gz


# FPKM Gene Expression with samtools and cufflinks

In order to generate the FPKM gene expression quantifications of the alignment result, we utilize cufflinks. However, since we are using **SAM** files, it needs to be converted first to **BAM** files, and samtools provide some convenient command to do so. Aside from the alignment files, we also need an additional annotated gene data, which is usually saved with gtf. format. The implementation of the package can be seen on the code below.

Additionally, we can directly generates FPKM based on the raw count we have.Using subreads, we generate the SAM files and convert it to the raw counts. After we are able to generate raw counts, we can calculate the FPKM based on the formula below:



```
FPKM = (Number of Fragments Mapped to Gene) / (Gene Length in Kilobases * Total Number of Fragments Mapped to All Genes in Millions)'
```





In [ ]:
def counting_raw_count(gtf_file, output_dir, sam_dir, file_pairs, program_path):

  os.makedirs(output_dir, exist_ok=True)

  # Define SAM file paths for forward and reverse reads
  forward_sam_file = os.path.join(sam_dir, file_pairs[0])
  reverse_sam_file = os.path.join(sam_dir, file_pairs[1])

  name = file_pairs[0].split('_')[0]

  # Output file name
  output_combined_counts = name + '.txt'

  # Run featureCounts for both forward and reverse reads
  featurecounts_command = (
      f"{program_path} -a {gtf_file} "
      f"-o {output_combined_counts} "
      f"-F GTF -p -B -C -f -T 4 -g ID "
      f"{forward_sam_file} {reverse_sam_file}"
  )
  os.system(featurecounts_command)

  print("FeatureCounts completed for both forward and reverse reads.")

def remove_duplicates(input_sam, output_sam, filename):
  # split name with type
  name = filename.split('.')[0]

  # create input and output
  input_sam_name = os.path.join(input_sam, filename)
  output_sam_name = os.path.join(output_sam, f"{name}_noduplicates.sam")


  # remove dupes
  !samtools rmdup {input_sam_name} {output_sam_name}
  print("duplicates removed.")


def FPKM_cufflink(forward_path, reverse_path, save_path):

  !samtools view -bS -o forward.bam {forward_path}
  !samtools view -bS -o reverse.bam {reverse_path}

  # Sort and index BAM files
  print('processing forward bam file')
  !samtools sort -o forward_sorted.bam forward.bam
  !samtools index forward_sorted.bam

  print("processing reverse bam file")
  !samtools sort -o reverse_sorted.bam reverse.bam
  !samtools index reverse_sorted.bam

  # Run Cufflinks separately on each strand
  print("run cufflink")
  !cufflinks -p 4 -o forward_output --library-type fr-firststrand forward_sorted.bam
  !cufflinks -p 4 -o reverse_output --library-type fr-secondstrand reverse_sorted.bam

  # Merge assemblies
  print("merge output")
  !cuffmerge -o merged_output assemblies.txt

  # Run Cuffquant on merged assemblies and original BAM files
  print('merge assemblies')
  !cuffquant -p 4 -o quantification_output merged_output/merged.gtf forward_sorted.bam
  !cuffquant -p 4 -o quantification_output merged_output/merged.gtf reverse_sorted.bam

  # Run Cuffnorm to normalize expression values
  print("normalize expression values")
  !cuffnorm -p 4 -o cuffnorm_output merged_output/merged.gtf quantification_output/forward_sorted/abundances.cxb quantification_output/reverse_sorted/abundances.cxb

  print('saving data')
  !cp cuffnorm_output/genes.fpkm_table.csv {save_path}

  print('data saved!')


def raw_counts_to_fpkm(raw_counts_file, output_csv_file):
    # Read raw counts from the FeatureCounts output file
    raw_counts_df = pd.read_csv(raw_counts_file, sep=' ', index_col=0, skiprows=1)

    # Calculate total mapped reads
    total_mapped_reads = raw_counts_df.sum(axis=0).sum()

    # Calculate feature lengths from raw counts
    feature_lengths = raw_counts_df.sum(axis=0).astype(str)

    # Calculate FPKM values
    fpkm_df = (raw_counts_df / feature_lengths.astype(float)) / (total_mapped_reads / 1e6)

    # Save FPKM values to a CSV file
    fpkm_df.to_csv(output_csv_file)


In [ ]:
# remove dupes
file_name = r"SRR1973492_2_alignment.sam"
input_sam = '/content/drive/MyDrive/fastaq/D_sim_jp_aligned'
output_sam =  '/content/drive/MyDrive/fastaq/removed_dup_d_sim_jp'

remove_duplicates(input_sam, output_sam, file_name)

In [ ]:
gtf_files = r'/content/drive/MyDrive/fastaq/reference_genome/dsim-all-r1.3.gff.gz'
output_dir = r'/content/drive/MyDrive/fastaq/reads'
file_pairs = ["SRR1973492_1_alignment.sam", "SRR1973492_2_alignment.sam"]
sam_dir = r'/content/drive/MyDrive/fastaq/D_sim_jp_aligned'

# need to declare the path first
subread_path = r'/usr/local/envs/featurecounts/bin/featureCounts'

counting_raw_count(gtf_files, output_dir, sam_dir,file_pairs, subread_path)

In [ ]:
raw_counts_file_path = '/content/drive/MyDrive/academic/bioinf/dataset/SRR1973492.txt'  # Replace with your actual raw counts file path
output_csv_file_path = '/content/drive/MyDrive/academic/bioinf/dataset/simjp_fpkm_values.csv'  # Replace with your desired output CSV file path

raw_counts_to_fpkm(raw_counts_file_path, output_csv_file_path)

# Normalization across samples

Since we have cross-samples (D. sech JP and D. sech TW), normalization technique needs to be used. In this case, we follow the settings explained in the paper, which is to utilize **q= 0.95** for upper quartile normalization. Before normalization, we change the gene names based on the OGS.

In our case, we change the NOISeq package to Numpy for convenience, since numpy is a python package.

In [ ]:
# Load the orthologous gene list from the document
ogs_gene = pd.read_csv('/content/drive/MyDrive/academic/bioinf/dataset/OGS_GENES_3_SPECIES.csv')

# Load FPKM Data for each species


# species name order should be melan, sech, sech, sim
def switch_gene_names(path_source, path_dest, ogs_gene, species_name=list()):

  species_name = sorted(species_name)
  fpkm_species1 = pd.read_csv(os.path.join(path_source, species_name[0]),delimiter='\t', skiprows=0)
  fpkm_species2 = pd.read_csv(os.path.join(path_source, species_name[1]),delimiter='\t', skiprows=0)
  fpkm_species3 = pd.read_csv(os.path.join(path_source, species_name[3]),delimiter='\t', skiprows=0)
  fpkm_species4 = pd.read_csv(os.path.join(path_source, species_name[4]),delimiter='\t', skiprows=0)

  ogs_gene.rename(columns={'#gene': 'Geneid'}, inplace=True)
  ogs_melan = ogs_gene[['Geneid', 'Dmel']].copy()
  ogs_sech = ogs_gene[['Geneid', 'Dsec']].copy()
  ogs_sim = ogs_gene[['Geneid', 'Dsim']].copy()

  melan_dict = ogs_melan.set_index('Dmel')['Geneid'].to_dict()
  sech_dict = ogs_sech.set_index('Dsec')['Geneid'].to_dict()
  sim_dict = ogs_sim.set_index('Dsim')['Geneid'].to_dict()

  fpkm_species1['Geneid'] = fpkm_species1['Geneid'].map(melan_dict).fillna(fpkm_species1['Geneid'])
  fpkm_species2['Geneid'] = fpkm_species2['Geneid'].map(sech_dict).fillna(fpkm_species2['Geneid'])
  fpkm_species3['Geneid'] = fpkm_species3['Geneid'].map(sech_dict).fillna(fpkm_species3['Geneid'])
  fpkm_species4['Geneid'] = fpkm_species4['Geneid'].map(sim_dict).fillna(fpkm_species4['Geneid'])

  # Save the merged data as OGS CSV files
  fpkm_species1.to_csv(os.path.join(path_dest, 'ogs_' + species_name[0]), index=False)
  fpkm_species2.to_csv(os.path.join(path_dest, 'ogs_' + species_name[1]), index=False)
  fpkm_species3.to_csv(os.path.join(path_dest, 'ogs_' + species_name[2]), index=False)
  fpkm_species4.to_csv(os.path.join(path_dest, 'ogs_' + species_name[3]), index=False)

In [ ]:
import numpy as np
import pandas as pd

def upper_quartile_norm(path_source, path_dest, filename, q=95, combine=True):

  # Load the gene expression counts table
  if combine == True:
    D_sech_tw = pd.read_csv(os.path.join(path_source, filename[0]), index_col=0)
    D_sech_jp = pd.read_csv(os.path.join(path_source, filename[1]), index_col=0)
    df = pd.concat([D_sech_tw, D_sech_jp], axis=0)

  else:
    df = pd.read_csv(os.path.join(path_source, filename), index_col=0)

  # Define the columns for normalization
  df['expression_column'] = df['FPKM']
  fpkm = np.array(df['expression_column'])

  # Calculate the upper quartile for each gene
  upper_quartile = np.percentile(fpkm, 95)

  # # # Normalize the gene expression data
  # df["expression_columns"] = df["expression_columns"].div(upper_quartile, axis=1)

  return df

  # # Save the normalized counts table
  # df.to_csv(os.path.join(path_dest,'normalized_counts_table_{name}.csv'))


In [ ]:
path_source= r'/content/drive/MyDrive/fastaq/ogs'
path_dest = r'/content/drive/MyDrive/fastaq/ogs_norm'
filename = ['ogs_sechtw.csv', 'ogs_sechjp.csv']

df = upper_quartile_norm(path_source, path_dest, filename, q=95, combine=True)
df

# creating the table and figures of upregulated and downregulated genes

incoming

In [ ]:
list_of_upregulated_genes = ['Obp19a', 'Obp50a', 'Obp56d', 'CheA75a', 'CheA87a', 'Or23a',
                             'Or35a', 'Or56a', 'Or67b', 'Or85b', 'Or85c', 'Gr64f', 'Ir84a']

list_of_downregulated_genes = ['Obp83a', 'Obp99c', 'Obp99d', 'Or9a', 'Or42b']


# Expression Divergance

We generated heat maps for the Or , Ir , Obp , and Gr gene families and also the whole antennal transcriptome for each sex in each species ( figs. 2–4 and supplementary figs. S2–S3 , Supplementary Material online). We calculated the average expression level (without upper quartile normalization) of each gene estimated from three sequencing lanes followed by ranking within each of the above four gene families. Duplicate genes excluded in the OGS were now included for expression profile analysis. Genes with lowest expression levels (FPKM = 0 in all the samples) were ranked as 1 (colored yellow in the heat maps), whereas those with highest expression levels were colored red in the heat maps. Hierarchical agglomerative clustering was applied to cluster species as well as genes for each gene family. The significance of clustering was tested by multiscale bootstrap resampling implemented in the pvclust R package ( Suzuki and Shimodaira 2006 ).

#